Парсинг lifehacker.ru раздел "Технологии"

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

##1. Пагинация

На сайте выбранный раздел имеет ссылку https://lifehacker.ru/topics/technology/

При смене страницы в адресной строке браузера появляется дополнительный параметр page, значение которого отвечает за текущую страницу: https://lifehacker.ru/topics/technology/?page=1

Таким образом, первые 10 страниц могут быть получены изменением значения этого параметра от 1 до 10, включительно.

##2. Разметка страниц

На каждой странице ссылка на статью сидит в html-элементе с тэгом a и классом "lh-small-article-card__link".

In [17]:
base_url = 'https://lifehacker.ru'

In [18]:
article_urls = []

In [19]:
for page_num in range(1, 11):
  url = f'{base_url}/topics/technology/?page={page_num}'
  responce = requests.get(url)
  soup = BeautifulSoup(responce.text, 'lxml')
  raw_items = soup.find_all('a', class_='lh-small-article-card__link')
  article_urls += [item.get('href') for item in raw_items]

In [20]:
len(article_urls)

300

In [59]:
article_urls[155:165]

['/arc-search-na-android/',
 '/ps5-vs-ps5-pro-za-i-protiv/',
 'https://lifehacker.ru/special/ora/',
 '/analogi-chatgpt/',
 '/chatgpt-canvas/',
 '/obzor-huawei-watch-d2-chasov-s-izmereniem-davleniya/',
 '/poisk-po-video-v-google-obektive/',
 '/anons-anker-soundcore-liberty-4-pro/',
 '/prilozheniya-android-sentyabr-2024/',
 '/gostevoi-rezhim-v-iphone/']

В списке присутствуют ссылки на рекламу, которая появляется на страницах, удалим эти ссылки.

In [60]:
new_article_urls = article_urls.copy()
article_urls.clear()
for url in new_article_urls:
  if 'https' not in url:
    article_urls.append(url)

In [61]:
len(article_urls)

298

In [62]:
article_urls[155:165]

['/arc-search-na-android/',
 '/ps5-vs-ps5-pro-za-i-protiv/',
 '/analogi-chatgpt/',
 '/chatgpt-canvas/',
 '/obzor-huawei-watch-d2-chasov-s-izmereniem-davleniya/',
 '/poisk-po-video-v-google-obektive/',
 '/anons-anker-soundcore-liberty-4-pro/',
 '/prilozheniya-android-sentyabr-2024/',
 '/gostevoi-rezhim-v-iphone/',
 '/kak-vybrat-saundbar/']

Получили массив со ссылками на все статьи первых 10 страниц. Теперь рассмотрим разметку страниц статьи, чтобы понять, как достать название статьи и текст.

Исходя из данных первой статьи, название сидит в тэге h1 с классом 'article-card__title'. Текст же сидит в тэге article с классом 'single-article__content-container'.

Проверим описанное выше на одной из статей:

In [37]:
article_url = 'https://lifehacker.ru/izmerenie-davleniya-v-apple-watch/'

In [38]:
test_responce = requests.get(article_url)
test_soup = BeautifulSoup(test_responce.text, 'lxml')
test_soup.find('h1', class_='article-card__title').text

'Будущие часы Apple Watch могут иметь датчики в ремешке для отслеживания давления'

In [39]:
test_soup.find('article').text.replace('\n', '').replace('\xa0', '')

'Apple, похоже, планирует оснастить ремешки будущих Apple Watch датчиками для измерения артериального давления. Похожая идея уже реализована в Huawei Watch D2.Согласно новому патенту компании, датчики могут быть встроены в эластичную тканевую ленту. Помимо давления они будут отслеживать дыхание и другие показатели здоровья.В документе также описывается технология, встроенная в ткань, которая может использоваться для беспроводного питания, что потенциально позволяет полностью удалённо заряжать носимое устройство.Ещё один вариант её применения: беспроводное взаимодействие с внешними электронными устройствами. Возможно, для передачи и приёма информации с iPhone или Mac. Чем это будет отличаться от уже существующих способов, пока неизвестно.Apple считает, что эта технология применима даже в одежде, включая головные уборы и перчатки.Интересно, что упоминается «повязка на голову из эластичной ткани», что напоминает ремешок гарнитуры Vision Pro. Не исключено, что купертинцы хотят интегрироват

Если задавать класс тэга, то не находит атрибут text.

##3. Получение названий и текстов из всех статей с первых 10 страниц.

In [72]:
result = []
for url in tqdm(article_urls):
  article = {}
  article_url = f'{base_url}{url}'
  responce = requests.get(article_url)
  soup = BeautifulSoup(responce.text, 'lxml')


  article['title'] = soup.find('h1', class_='article-card__title').text
  article['text'] = soup.find('article').text.replace('\xa0', '').replace('\n', '')

  result.append(article)

100%|██████████| 298/298 [03:38<00:00,  1.36it/s]


##4. Создание датафрейма.

In [73]:
pd.set_option('display.max_colwidth', 400)

In [74]:
data = pd.DataFrame(result)
data.head()

,title,text
0,Будущие часы Apple Watch могут иметь датчики в ремешке для отслеживания давления,"Apple, похоже, планирует оснастить ремешки будущих Apple Watch датчиками для измерения артериального давления. Похожая идея уже реализована в Huawei Watch D2.Согласно новому патенту компании, датчики могут быть встроены в эластичную тканевую ленту. Помимо давления они будут отслеживать дыхание и другие показатели здоровья.В документе также описывается технология, встроенная в ткань, которая мо..."
1,"PlayStation 5 Pro появилась на ‭«Яндекс Маркете» — дешевле, чем по предзаказу","Новая PlayStation 5 Pro, представленная в сентябре, появилась на ‭«Яндекс Маркете». Заказать консоль можно за 83 850 рублей. Также придётся заплатить пошлину за доставку из-за рубежа в размере 10 650 рублей. Итого общая стоимость покупки составит 94 500 рублей. Это значительно меньше, чем за предзаказ в крупных сетях электроники — там новинка стоит 124 990 рублей.В обоих случаях доставляется в..."
2,В «Яндекс Браузере» появилась возможность получать кешбэк рублями,"В «Яндекс Браузере» теперь можно получать кешбэк рублями за покупки на сайтах и выводить эти деньги на карту. Максимальный кешбэк — 30%.Функция доступна в магазинах и на маркетплейсах. Среди них — «Яндекс Маркет», «ВсеИнструменты.ру» и Hoff — в общей сложности более 350 площадок, и список будет пополняться.Изображение: ЯндексДля получения кешбэка нужно авторизоваться в браузере. Если на сайте ..."
3,"8 гаджетов и бытовых приборов, о покупке которых жалеет редакция Лайфхакера",Оглавление Умные весы Беспроводной пылесос Фен-стайлер Паровая швабра Пауэрбанк с грелкой для рук Ручной отпариватель для одежды Ультразвуковая стиральная машинка Аэрогриль Оглавление Умные весы ...
4,Samsung Galaxy Ring 2 может выйти раньше и с рядом полезных изменений,"Умное кольцо Galaxy Ring от Samsung вышло этим летом, поэтому предполагалось, что его новая версия будет представлена летом следующего года. Однако, согласно корейскому инсайдеру, анонс может состояться раньше.К сожалению, насколько раньше, источник не рассказал, поэтому о дате релиза можно только гадать. В январе должен выйти Galaxy S25, и Samsung вместе с линейкой смартфонов могла бы предста..."
